Data yang digunakan pada model ini adalah data dummy, karena data asli bersifat private

In [ ]:
pip install pmdarima

In [ ]:
# Library
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, LSTM, Input
from pmdarima.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

# Function

In [ ]:
# utils
def mean_absolute_percentage_error(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# convert an array of values into a dataset matrix
def split_sequence(sequence, n_steps=1):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# make a forecast
def forecast(model, history, n_steps):
	# flatten data
	data = np.array(history)
	# retrieve last observations for input data
	input_x = data[-n_steps:, :]
	# reshape into [1, n_input, p]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

# Import Dataset

In [ ]:
my_data = pd.read_csv('/content/aftersidang dummy 2.csv')
my_data.set_index('Date', inplace=True)

In [ ]:
my_data.head()

,Gentamicin salep mata,Gentamicin tetes mata,Gentian violet larutan 1%,Glibenclamid 5 mg,Gliserin,Griseofulvin 125 mg,"Haloperidol 0,5 mg","Haloperidol 1,5 mg",Hidrokortison SK,Ibuprofen 200 mg,...,Ispa,Commoncold,Hypertensi,Cepalgia,Dermatitis,Otitis arthritis,Myalgia,Gimul,Konjungtivitis,Dyspebsia
Date,,,,,,,,,,,,,,,,,,,,,
1/1/2016,0,3,0,48,0,93,0,0,3,1371,...,327,275,207,164,200,81,163,157,64,215
4/1/2016,0,3,0,47,0,103,0,0,3,1521,...,373,297,230,237,250,73,209,185,33,252
7/1/2016,0,3,0,50,0,84,0,0,3,1228,...,443,322,299,158,262,158,198,239,97,250
10/1/2016,0,3,0,46,0,93,0,0,3,1322,...,386,320,256,146,236,123,219,211,53,227
1/1/2017,0,3,0,47,0,104,0,0,3,1564,...,310,269,209,187,200,146,155,177,60,193


In [ ]:
my_data = my_data.astype("float32")

In [ ]:
my_data.shape

(30, 90)

In [ ]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 1/1/2016 to 4/1/2023
Data columns (total 90 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gentamicin salep mata              30 non-null     float32
 1   Gentamicin tetes mata              30 non-null     float32
 2   Gentian violet larutan 1%          30 non-null     float32
 3   Glibenclamid 5 mg                  30 non-null     float32
 4   Gliserin                           30 non-null     float32
 5   Griseofulvin 125 mg                30 non-null     float32
 6   Haloperidol 0,5 mg                 30 non-null     float32
 7   Haloperidol 1,5 mg                 30 non-null     float32
 8   Hidrokortison SK                   30 non-null     float32
 9   Ibuprofen 200 mg                   30 non-null     float32
 10  Ibupropen 400 mg                   30 non-null     float32
 11  Ibupropen syrup                    30 non-null     f

In [ ]:
my_data

,Gentamicin salep mata,Gentamicin tetes mata,Gentian violet larutan 1%,Glibenclamid 5 mg,Gliserin,Griseofulvin 125 mg,"Haloperidol 0,5 mg","Haloperidol 1,5 mg",Hidrokortison SK,Ibuprofen 200 mg,...,Ispa,Commoncold,Hypertensi,Cepalgia,Dermatitis,Otitis arthritis,Myalgia,Gimul,Konjungtivitis,Dyspebsia
Date,,,,,,,,,,,,,,,,,,,,,
1/1/2016,0.0,3.0,0.0,48.0,0.0,93.0,0.0,0.0,3.0,1371.0,...,327.0,275.0,207.0,164.0,200.0,81.0,163.0,157.0,64.0,215.0
4/1/2016,0.0,3.0,0.0,47.0,0.0,103.0,0.0,0.0,3.0,1521.0,...,373.0,297.0,230.0,237.0,250.0,73.0,209.0,185.0,33.0,252.0
7/1/2016,0.0,3.0,0.0,50.0,0.0,84.0,0.0,0.0,3.0,1228.0,...,443.0,322.0,299.0,158.0,262.0,158.0,198.0,239.0,97.0,250.0
10/1/2016,0.0,3.0,0.0,46.0,0.0,93.0,0.0,0.0,3.0,1322.0,...,386.0,320.0,256.0,146.0,236.0,123.0,219.0,211.0,53.0,227.0
1/1/2017,0.0,3.0,0.0,47.0,0.0,104.0,0.0,0.0,3.0,1564.0,...,310.0,269.0,209.0,187.0,200.0,146.0,155.0,177.0,60.0,193.0
4/1/2017,0.0,3.0,0.0,49.0,0.0,113.0,0.0,0.0,3.0,1679.0,...,230.0,237.0,192.0,209.0,143.0,94.0,193.0,154.0,90.0,187.0
7/1/2017,0.0,3.0,0.0,50.0,0.0,108.0,0.0,0.0,3.0,1888.0,...,421.0,283.0,265.0,229.0,222.0,73.0,234.0,179.0,129.0,152.0
10/1/2017,0.0,3.0,0.0,51.0,0.0,103.0,0.0,0.0,4.0,1846.0,...,484.0,38.0,316.0,251.0,243.0,14.0,261.0,245.0,12.0,252.0
1/1/2018,0.0,4.0,0.0,52.0,0.0,105.0,0.0,0.0,7.0,1842.0,...,495.0,117.0,228.0,256.0,284.0,77.0,300.0,286.0,75.0,263.0


# Preprocessing Data

In [ ]:
my_data.describe()

,Gentamicin salep mata,Gentamicin tetes mata,Gentian violet larutan 1%,Glibenclamid 5 mg,Gliserin,Griseofulvin 125 mg,"Haloperidol 0,5 mg","Haloperidol 1,5 mg",Hidrokortison SK,Ibuprofen 200 mg,...,Ispa,Commoncold,Hypertensi,Cepalgia,Dermatitis,Otitis arthritis,Myalgia,Gimul,Konjungtivitis,Dyspebsia
count,30.0,30.000000,30.0,30.000000,30.0,30.000000,30.0,30.000000,30.000000,30.000000,...,30.000000,30.000000,30.000000,30.00000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.0,10.533334,0.0,97.133331,0.0,124.666664,0.0,0.066667,28.733334,2249.966553,...,315.899994,233.033340,214.699997,182.03334,201.566666,94.566666,190.533340,178.066666,85.000000,184.033340
std,0.0,5.727931,0.0,64.807800,0.0,30.802299,0.0,0.253708,18.913475,794.990479,...,118.453934,76.612091,64.716438,56.38261,47.728893,47.125427,60.922638,52.590767,46.489155,63.271984
min,0.0,3.000000,0.0,45.000000,0.0,84.000000,0.0,0.000000,3.000000,1228.000000,...,160.000000,38.000000,107.000000,90.00000,124.000000,14.000000,45.000000,98.000000,12.000000,44.000000
25%,0.0,3.250000,0.0,48.250000,0.0,103.000000,0.0,0.000000,4.750000,1646.750000,...,231.250000,202.500000,170.500000,138.50000,163.250000,74.000000,164.000000,142.000000,56.250000,137.750000
50%,0.0,12.500000,0.0,51.500000,0.0,113.500000,0.0,0.000000,36.000000,1926.500000,...,283.000000,222.500000,208.000000,181.00000,200.000000,78.500000,194.000000,169.500000,75.000000,185.500000
75%,0.0,16.000000,0.0,130.000000,0.0,143.500000,0.0,0.000000,45.000000,2870.000000,...,407.750000,280.250000,263.500000,213.50000,242.500000,118.750000,225.000000,221.250000,99.250000,226.000000
max,0.0,18.000000,0.0,241.000000,0.0,189.000000,0.0,1.000000,49.000000,3638.000000,...,589.000000,468.000000,322.000000,335.00000,287.000000,248.000000,310.000000,286.000000,231.000000,312.000000


In [ ]:
# seperate y and exogenous X
Y = my_data.iloc[:,0:80]
X = my_data.iloc[:,80:]

In [ ]:
X

,Ispa,Commoncold,Hypertensi,Cepalgia,Dermatitis,Otitis arthritis,Myalgia,Gimul,Konjungtivitis,Dyspebsia
Date,,,,,,,,,,
1/1/2016,327.0,275.0,207.0,164.0,200.0,81.0,163.0,157.0,64.0,215.0
4/1/2016,373.0,297.0,230.0,237.0,250.0,73.0,209.0,185.0,33.0,252.0
7/1/2016,443.0,322.0,299.0,158.0,262.0,158.0,198.0,239.0,97.0,250.0
10/1/2016,386.0,320.0,256.0,146.0,236.0,123.0,219.0,211.0,53.0,227.0
1/1/2017,310.0,269.0,209.0,187.0,200.0,146.0,155.0,177.0,60.0,193.0
4/1/2017,230.0,237.0,192.0,209.0,143.0,94.0,193.0,154.0,90.0,187.0
7/1/2017,421.0,283.0,265.0,229.0,222.0,73.0,234.0,179.0,129.0,152.0
10/1/2017,484.0,38.0,316.0,251.0,243.0,14.0,261.0,245.0,12.0,252.0
1/1/2018,495.0,117.0,228.0,256.0,284.0,77.0,300.0,286.0,75.0,263.0


In [ ]:
Y

,Gentamicin salep mata,Gentamicin tetes mata,Gentian violet larutan 1%,Glibenclamid 5 mg,Gliserin,Griseofulvin 125 mg,"Haloperidol 0,5 mg","Haloperidol 1,5 mg",Hidrokortison SK,Ibuprofen 200 mg,...,Vit. B 12,Vit. B 6,Vit. B Complex,Vit. C 10 mg,Vit. C 50 mg,Vit. C 500 mg,Vit. K,Whitfield,Yodium Povidon,Zinc 20 mg
Date,,,,,,,,,,,,,,,,,,,,,
1/1/2016,0.0,3.0,0.0,48.0,0.0,93.0,0.0,0.0,3.0,1371.0,...,73.0,14.0,634.0,0.0,619.0,0.0,0.0,0.0,2.0,820.0
4/1/2016,0.0,3.0,0.0,47.0,0.0,103.0,0.0,0.0,3.0,1521.0,...,73.0,14.0,624.0,0.0,715.0,0.0,0.0,0.0,2.0,814.0
7/1/2016,0.0,3.0,0.0,50.0,0.0,84.0,0.0,0.0,3.0,1228.0,...,75.0,15.0,658.0,0.0,557.0,0.0,0.0,0.0,2.0,826.0
10/1/2016,0.0,3.0,0.0,46.0,0.0,93.0,0.0,0.0,3.0,1322.0,...,71.0,14.0,607.0,0.0,554.0,0.0,0.0,0.0,2.0,816.0
1/1/2017,0.0,3.0,0.0,47.0,0.0,104.0,0.0,0.0,3.0,1564.0,...,73.0,14.0,638.0,0.0,746.0,0.0,0.0,0.0,3.0,818.0
4/1/2017,0.0,3.0,0.0,49.0,0.0,113.0,0.0,0.0,3.0,1679.0,...,75.0,13.0,629.0,0.0,844.0,0.0,0.0,0.0,3.0,808.0
7/1/2017,0.0,3.0,0.0,50.0,0.0,108.0,0.0,0.0,3.0,1888.0,...,76.0,15.0,631.0,0.0,841.0,0.0,0.0,0.0,3.0,785.0
10/1/2017,0.0,3.0,0.0,51.0,0.0,103.0,0.0,0.0,4.0,1846.0,...,86.0,27.0,689.0,0.0,816.0,0.0,0.0,0.0,3.0,763.0
1/1/2018,0.0,4.0,0.0,52.0,0.0,105.0,0.0,0.0,7.0,1842.0,...,108.0,26.0,680.0,0.0,813.0,0.0,0.0,0.0,2.0,728.0


# MODEL

# define model 5
def LSTM_Model(input_Shape):

    inputs = Input(input_Shape)

    x = LSTM(64, return_sequences=True)(inputs)
    x = LSTM(64, return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs = inputs, outputs = x)
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

# define model 2
def LSTM_Model(input_Shape):

    inputs = Input(input_Shape)

    x = LSTM(256, return_sequences=True)(inputs)
    x = Dropout(0.2)(x)
    x = LSTM(128)(x)
    x = Dropout(0.1)(x)
    x = Dense(128)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs = inputs, outputs = x)
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

# define model 1
def LSTM_Model(input_Shape):

    inputs = Input(input_Shape)

    x = LSTM(32, return_sequences=True)(inputs)
    x = Dropout(0.2)(x)
    x = LSTM(16)(x)
    x = Dropout(0.1)(x)
    x = Dense(1)(x)

    model = Model(inputs = inputs, outputs = x)
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

# DEFINE MODEL

In [ ]:
# define model 2
def LSTM_Model(input_Shape):

    inputs = Input(input_Shape)

    x = LSTM(256, return_sequences=True)(inputs)
    x = Dropout(0.2)(x)
    x = LSTM(128)(x)
    x = Dropout(0.1)(x)
    x = Dense(128)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs = inputs, outputs = x)
    model.compile(loss='mean_squared_error', optimizer='Adam')

    return model

# Modelling And Evaluation

In [ ]:
import os

# Buat direktori bernama `images`
os.mkdir("images")

In [ ]:
# random seeds
def reset_seeds():
   np.random.seed(333)
   random.seed(333)
   tf.random.set_seed(333)

reset_seeds()

# define container
mymodel = []
evaluation = []
forcasting = []

# looping for all Y dependents
for index, values in enumerate(Y.columns):
    # concatenate
    combined = pd.concat([X, Y[[values]]], axis=1).values

    # scale data
    scaler = MinMaxScaler(feature_range=(0,1)).fit(combined)
    scaled_combined = scaler.transform(combined)

    #split data into train and training set (70:30)
    scaled_train_combined, scaled_test_combined  = train_test_split(scaled_combined, test_size=0.30)

    # reshape X and Y
    n_steps = 1 # number of time lag
    trainX, trainY = split_sequence(scaled_train_combined, n_steps)
    testX, testY = split_sequence(scaled_test_combined, n_steps)

    # fit LSTM network
    model = LSTM_Model((trainX.shape[1],trainX.shape[2]))
    hist = model.fit(trainX, trainY[:,-1].reshape(-1,1), epochs=100, batch_size=2,
                    validation_data=(testX, testY[:,-1].reshape(-1,1)), verbose=0, shuffle=False)
    mymodel.append(model)

    # plot performance
    fig = plt.figure(figsize=(16,10), dpi=600)
    plt.plot(hist.history['loss'], color= 'black', scaley=1)
    plt.plot(hist.history['val_loss'], color='gold')
    plt.title(f'Model Loss for {values}')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.ylim(0, 1)
    plt.savefig(f'images/model_loss_{values}.png')
    plt.close(fig)

    # make a prediction for train set
    yhat = np.concatenate((scaled_combined[:n_steps, -1].reshape(-1,1), model.predict(trainX, verbose=0)))

    # make a prediction for test set
    history = [x for x in scaled_train_combined]
    # walk-forward forecasting
    predictions = list()
    for i in range(len(scaled_test_combined)):
        # predict
        yhat_sequence = forecast(model, history, n_steps)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting
        history.append(scaled_test_combined[i,:])

    # evaluate forecasts
    predic = np.concatenate((yhat, predictions))
    lstm_preds = scaler.inverse_transform(np.concatenate((scaled_combined[:, :-1], predic), axis=1))
    rmse = math.sqrt(mean_squared_error(combined[1:,-1], lstm_preds[1:,-1]))
    mape = mean_absolute_percentage_error(combined[1:,-1], lstm_preds[1:,-1])
    rsq = np.corrcoef(combined[1:,-1], lstm_preds[1:,-1])[0,1]**2
    evaluation.append({"Outputs": values, "RMSE": rmse, "MAPE": mape, "R Square": rsq})
    forcasting.append(lstm_preds[:,-1])

    print(f'Evaluation Result for {values} :')
    print('RMSE : %.3f' % rmse)
    print('MAPE : %.3f' % mape)
    print('R Square : %.3f' % rsq)

    # plot forecasts against actual outcomes
    fig = plt.figure(figsize=(16,10), dpi=600)
    plt.plot(axis='Y[values].index', labelrotation=90)
    plt.plot(Y[values].index[1:], Y[values].values[1:], color = 'gold', label=f'Actual {values}')
    plt.plot(Y[values].index[1:], lstm_preds[1:,-1], color = 'black',label='Predicted train set')
    plt.plot(Y[values].index[-8:], lstm_preds[-8:,-1], color = 'red',label='Predicted test set')
    plt.xticks(Y[values].index, rotation='vertical')
    plt.xlabel('Periode Waktu')
    plt.ylabel(f'{values}')
    plt.legend(loc='upper left', fontsize=8)
    plt.savefig(f'images/Prediction_{values}.png')
    plt.close(fig)

    # Lstm Feature Importance
    X_valid = trainX.copy()
    results = []

    oof_preds = model.predict(X_valid, verbose=0).squeeze()
    baseline_mae = np.mean(np.abs(oof_preds-trainY[:,-1]))
    results.append({'feature':'BASELINE','mae': baseline_mae})

    for k in range(len(X.columns)):
        # shuffle feature k
        save_col = X_valid[:,:,k].copy()
        np.random.shuffle(X_valid[:,:,k])

        # Compute OOF MAE With Feature K Shuffled
        oof_preds = model.predict(X_valid, verbose=0).squeeze()
        mae = np.mean(np.abs(oof_preds-trainY[:,-1]))
        results.append({'feature': X.columns[k],'mae': mae})
        X_valid[:,:,k] = save_col

    # Display Lstm Feature Importance
    df = pd.DataFrame(results)
    df = df.sort_values('mae')
    fig = plt.figure(figsize=(7,5))
    plt.barh(np.arange(len(X.columns) + 1),df.mae)
    plt.yticks(np.arange(len(X.columns)+1),df.feature.values)
    plt.title(f'LSTM Feature Importance for {values}')
    plt.ylim((-1,len(X.columns)+1))
    plt.plot([baseline_mae,baseline_mae],[-1,len(X.columns)+1], '--', color='orange',
                label=f'Baseline OOF\n MAE={baseline_mae:.3f}')
    plt.xlabel(f'Permutation Feature Importance')
    plt.ylabel('Feature')
    plt.legend()
    plt.savefig(f'images/Feature Importance_{values}.png', bbox_inches='tight')
    plt.close(fig)

Evaluation Result for Gentamicin salep mata :
RMSE : 0.006
MAPE : inf
R Square : nan
Evaluation Result for Gentamicin tetes mata :
RMSE : 0.837
MAPE : 7.068
R Square : 0.983
Evaluation Result for Gentian violet larutan 1% :
RMSE : 0.006
MAPE : inf
R Square : nan
Evaluation Result for Glibenclamid 5 mg :
RMSE : 17.073
MAPE : 8.085
R Square : 0.957
Evaluation Result for Gliserin :
RMSE : 0.006
MAPE : inf
R Square : nan
Evaluation Result for Griseofulvin 125 mg :
RMSE : 12.571
MAPE : 7.058
R Square : 0.870
Evaluation Result for Haloperidol 0,5 mg :
RMSE : 0.006
MAPE : inf
R Square : nan
Evaluation Result for Haloperidol 1,5 mg :
RMSE : 0.262
MAPE : inf
R Square : 0.001
Evaluation Result for Hidrokortison SK :
RMSE : 1.440
MAPE : 10.807
R Square : 0.997
Evaluation Result for Ibuprofen 200 mg :
RMSE : 206.462
MAPE : 8.473
R Square : 0.930
Evaluation Result for Ibupropen 400 mg :
RMSE : 133.530
MAPE : 3.340
R Square : 0.933
Evaluation Result for Ibupropen syrup :
RMSE : 1.244
MAPE : 20.609
R

In [ ]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
)

In [ ]:
pip install xlsxwriter

In [ ]:
# save forecasting & evaluation result
writer = pd.ExcelWriter("evaluation output.xlsx", engine = 'xlsxwriter')
df1 = pd.DataFrame(np.array(forcasting).T, index= Y.index, columns = Y.columns)
df2 = pd.DataFrame(evaluation)
df1.to_excel(writer, sheet_name='forecasting')
df2.to_excel(writer, sheet_name='evaluation', index=False)
writer.close()

In [ ]:
!zip -r /content/images.zip /content/images

updating: content/images/ (stored 0%)
updating: content/images/model_loss_Domperidone Sirup.png (deflated 57%)
updating: content/images/model_loss_Nistatin tablet.png (deflated 54%)
updating: content/images/Feature Importance_Amitriftilin 25 mg.png (deflated 13%)
updating: content/images/Feature Importance_Hidrokortison SK.png (deflated 13%)
updating: content/images/Feature Importance_Carbamazepine .png (deflated 13%)
updating: content/images/Feature Importance_Kalsium Laktat.png (deflated 13%)
updating: content/images/Feature Importance_New Antides.png (deflated 13%)
updating: content/images/Feature Importance_Tablet tambah darah.png (deflated 13%)
updating: content/images/model_loss_Kalsium Laktat.png (deflated 54%)
updating: content/images/model_loss_Metronidazol 500 mg.png (deflated 57%)
updating: content/images/model_loss_SF.png (deflated 53%)
updating: content/images/Feature Importance_Metronidazol 500 mg.png (deflated 13%)
updating: content/images/Feature Importance_Acetylsistei

In [ ]:
trainX.shape

(18, 3, 10)

In [ ]:
trainY.shape

(18, 10)

In [ ]:
testX.shape

(6, 3, 10)

In [ ]:
testY.shape

(6, 10)

In [ ]:
model.summary()

Model: "model_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_41 (InputLayer)       [(None, 3, 10)]           0         
                                                                 
 lstm_80 (LSTM)              (None, 3, 256)            273408    
                                                                 
 dropout_80 (Dropout)        (None, 3, 256)            0         
                                                                 
 lstm_81 (LSTM)              (None, 128)               197120    
                                                                 
 dropout_81 (Dropout)        (None, 128)               0         
                                                                 
 dense_80 (Dense)            (None, 128)               16512     
                                                                 
 dense_81 (Dense)            (None, 1)                 129

# WEIGHT AND BIAS

In [ ]:
lstm_layer = model.layers[1]

In [ ]:
kernel_initializer = lstm_layer.kernel_initializer

In [ ]:
config = kernel_initializer.get_config()

In [ ]:
print("Fungsi Inisialisasi Default untuk Bobot Kernel:", kernel_initializer.__class__.__name__)

Fungsi Inisialisasi Default untuk Bobot Kernel: GlorotUniform


In [ ]:
# Mendapatkan layer LSTM pertama
lstm_layer_1 = model.layers[1]

# Mendapatkan nilai default inisialisasi bobot kernel dan bias pada layer LSTM pertama
weights = lstm_layer_1.get_weights()

# Pisahkan bobot untuk masing-masing gate
W_i, W_f, W_c, W_o = weights[0][:, :128], weights[0][:, 128:256], weights[0][:, 256:384], weights[0][:, 384:]

# Pisahkan bias untuk masing-masing gate
b_i, b_f, b_c, b_o = weights[2][:128], weights[2][128:256], weights[2][256:384], weights[2][384:]

# Output nilai default inisialisasi untuk bobot setiap gate
print("Inisialisasi untuk Bobot Input Gate (W_i):", W_i)
print("Inisialisasi untuk Bobot Forget Gate (W_f):", W_f)
print("Inisialisasi untuk Bobot Cell Candidate (W_c):", W_c)
print("Inisialisasi untuk Bobot Output Gate (W_o):", W_o)

# Output nilai default inisialisasi untuk bias setiap gate
print("Inisialisasi untuk Bias Input Gate (b_i):", b_i)
print("Inisialisasi untuk Bias Forget Gate (b_f):", b_f)
print("Inisialisasi untuk Bias Cell Candidate (b_c):", b_c)
print("Inisialisasi untuk Bias Output Gate (b_o):", b_o)

Inisialisasi untuk Bobot Input Gate (W_i): [[ 0.01363827  0.1030093   0.11270316 ...  0.01655106  0.07542837
   0.07812592]
 [ 0.09158807  0.09476741 -0.03290947 ... -0.06503744 -0.03311546
   0.04073571]
 [-0.05351787  0.04343985  0.03946022 ...  0.02302325  0.09337191
   0.05154794]
 ...
 [-0.00572065 -0.06046853 -0.02939237 ... -0.01319833  0.00922244
   0.02223073]
 [-0.00137533  0.14744952  0.06971064 ...  0.01032614 -0.0964389
  -0.00656997]
 [ 0.02605655  0.12956825  0.02299381 ... -0.12229291 -0.15139852
  -0.00848454]]
Inisialisasi untuk Bobot Forget Gate (W_f): [[ 0.08557734  0.04830033  0.05890609 ...  0.17936403 -0.06047326
   0.13966952]
 [ 0.0187434   0.00195532 -0.01079542 ...  0.07057815 -0.0311568
   0.03222793]
 [ 0.03744486  0.14662059  0.07524452 ...  0.09792466 -0.09803066
   0.06586154]
 ...
 [ 0.0521574   0.03103612 -0.02102423 ...  0.1541817  -0.00644316
  -0.09416676]
 [ 0.08945248  0.09122352  0.02100342 ...  0.1121195   0.03720328
   0.02061693]
 [-0.11088507